Адаптируем модель KNN для задачи регресии на данных для курсового проекта

In [3]:
import numpy as np
import pandas as pd
from sklearn import model_selection

In [4]:
# Получим полные рабочие данные в нумпай
X = pd.read_csv('x_train.csv')
X_train=X.to_numpy()
X_test = pd.read_csv('x_test.csv')
y = pd.read_csv('y_train.csv')
y_train = y.to_numpy()

In [5]:
X_tr,X_te, y_tr,y_t = model_selection.train_test_split(X_train, y_train, test_size=0.3, random_state=1)

In [6]:
# клас для расчета веса по номеру соседа
class Knn_w_n():
    def __init__ (self):
        self.x_train=None
        self.y_train=None
        self.x_test=None
        self.k=None
        self.answer = None
        
    def e_metrics(self,x1, x2):   
        return np.sqrt(((x1-x2)**2).sum())  
    
    def predict(self,x_train, y_train, x_test, k,q=0.9):
        self.x_train=x_train
        self.y_train=y_train
        self.x_test=x_test
        self.k=k
        self.q = q
        answers = []
        for x in x_test:
            test_distances = []

            for i in range(len(x_train)):

                # расчет расстояния от классифицируемого объекта до
                # объекта обучающей выборки
                distance = self.e_metrics(x, x_train[i])

                # Записываем в список значение расстояния и ответа на объекте обучающей выборки
                test_distances.append((distance, y_train[i]))

            # создаем словарь со всеми возможными классами
#             classes = {class_item: 0 for class_item in set(y_train)}

#             создадим список весов
            w=[q**i for i in range(k)]
            # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов           
            s=0
            for i,d in enumerate(sorted(test_distances)[0:k]):
                s+=d[1]*w[i]         
            answers.append(s/sum(w))
        self.answer = answers
        return answers
    
    def accuracy(self,y):
        return (sum(self.answer == y) / len(y))
    
    def r2 (self,y_test):
         return 1- ((self.answer - y_test)**2).sum()/((y_test-y_test.mean())**2).sum()

In [7]:
# клас для расчета пос редней между соседями без учета весов
class Knn():
    def __init__ (self):
        self.x_train=None
        self.y_train=None
        self.x_test=None
        self.k=None
        self.answer = None
        
    def e_metrics(self,x1, x2):   
        return np.sqrt(((x1-x2)**2).sum())  
    
    def predict(self,x_train, y_train, x_test, k):
        self.x_train=x_train
        self.y_train=y_train
        self.x_test=x_test
        self.k=k  
        answers = []
        for x in x_test:
            test_distances = []

            for i in range(len(x_train)):

                # расчет расстояния от классифицируемого объекта до
                # объекта обучающей выборки
                distance = self.e_metrics(x, x_train[i])

                # Записываем в список значение расстояния и ответа на объекте обучающей выборки
                test_distances.append((distance, y_train[i]))

            # создаем словарь со всеми возможными классами
#             classes = {class_item: 0 for class_item in set(y_train)}

#             создадим список весов
#             w=[0.9**i for i in range(k)]
            # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов           
            s=0
            for d in sorted(test_distances)[0:k]:
                s+=d[1]         
            answers.append(s/k)
        self.answer = answers
        return answers
    
    def accuracy(self,y):
        return (sum(self.answer == y) / len(y))
    
    def r2 (self,y_test):
         return 1- ((self.answer - y_test)**2).sum()/((y_test-y_test.mean())**2).sum()

In [8]:
# клас для расчета пос редней между соседями без учета весов
class Knn_w_d():
    def __init__ (self):
        self.x_train=None
        self.y_train=None
        self.x_test=None
        self.k=None
        self.answer = None
        
    def e_metrics(self,x1, x2):   
        return np.sqrt(((x1-x2)**2).sum())  
    
    def predict(self,x_train, y_train, x_test, k,q=0.9):
        self.x_train=x_train
        self.y_train=y_train
        self.x_test=x_test
        self.k=k 
        self.q = q
        answers = []
        for x in x_test:
            test_distances = []

            for i in range(len(x_train)):

                # расчет расстояния от классифицируемого объекта до
                # объекта обучающей выборки
                distance = self.e_metrics(x, x_train[i])

                # Записываем в список значение расстояния и ответа на объекте обучающей выборки
                test_distances.append((distance, y_train[i]))

            # создаем словарь со всеми возможными классами
#             classes = {class_item: 0 for class_item in set(y_train)}

#             создадим список весов
#             w=[0.9**i for i in range(k)]
            # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов           
            s=0
            w = 0
            for d in sorted(test_distances)[0:k]:
                s+=d[1]*q**d[0]
                w+=q**d[0]
            answers.append(s/w)
        self.answer = answers
        return answers
    
    def accuracy(self,y):
        return (sum(self.answer == y) / len(y))
    
    def r2 (self,y_test):
         return 1- ((self.answer - y_test)**2).sum()/((y_test-y_test.mean())**2).sum()

In [9]:
def R2 (y_pred,y_test):
         return 1- ((y_pred - y_test)**2).sum()/((y_test-y_test.mean())**2).sum()

In [11]:
def testing_algoritms():
    test1 = Knn_w_n()
    test2 = Knn()
    test3=Knn_w_d()
    results  = pd.DataFrame(columns=['k','q','num','dist','mean'])
    for i in [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        for j in [10,15,20,25]:
            test1.predict(X_tr, y_tr, X_te, j,q=i)
            test2.predict(X_tr, y_tr, X_te, j)
            test3.predict(X_tr, y_tr, X_te, j,q=i)
            results.loc[len(results)]=[j,i,test1.r2(y_t),test3.r2(y_t),test2.r2(y_t)]
    print(results)
    return results 

In [ ]:
testing_algoritms()